# Explore the data

# Start simple. 
GOAL: get a good prediction as quickly as possible
Don'e do the "Hh  poverty score continuation' for now

In [5]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


In [6]:
# Additional covariate called family size. to USE more data. 
# How to use more data by groupping poverty_scores on household?
# within same household
# add a psu_hh uui as a VARIABLE to tell the model "predict something similar for members of the same family". It would be difficult, there is a lot.
# stacking? what is stacking?

# Root preprocessing:

In [8]:
train_ed = pd.read_csv('ed_train_raw.csv') # education
train_hh = pd.read_csv('hh_train_raw.csv') # household

test_ed = pd.read_csv('edu_test.csv')
test_hh = pd.read_csv('hh_test.csv')

poverty = pd.read_csv('poverty_train_raw.csv') # poverty/answers

In [9]:
### Prepare Ed data
def prepare_df(df, suffix:str):
    uids = df['psu'].astype(str) + "_"  + df['hh'].astype(str) + "_" + df['idcode'].astype(str) 
    df = df.drop(columns=['psu', 'hh', 'idcode'])
    df.columns = [suffix + "_" + col.capitalize() for col in df.columns]
    df.insert(0, 'uid', uids)
    return df

test_ed = prepare_df(test_ed, 'ED')
test_hh = prepare_df(test_hh, 'HH')
train_ed = prepare_df(train_ed, 'ED')
train_hh = prepare_df(train_hh, 'HH')

In [10]:
### compute poverty scores
for i in range(1,11):
    col = 'subjective_poverty_'+ str(i)
    poverty.loc[poverty[col]==1, 'poverty_score'] = i

poverty['uid'] = poverty['psu_hh_idcode']
poverty = poverty[['uid', 'poverty_score']]



# Filter data based on uids that have a poverty score assigned to them.

In [12]:
train_ed = train_ed[train_ed['uid'].isin(poverty['uid'])]
train_ed = train_ed.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')

train_hh = train_hh[train_hh['uid'].isin(poverty['uid'])]
train_hh = train_hh.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')

In [13]:
# quite a lot of data wasted... nothing we can do.... right?
print(train_ed.shape)
print(train_hh.shape)
print(poverty.shape)


(5334, 68)
(5334, 25)
(5337, 2)


In [14]:
# df_hh.to_csv('hh_train_transform.csv', index=False)
# df_ed.to_csv('ed_train_transform.csv', index=False)
# df_pov_scores.to_csv('scores_train_transform.csv', index=False)


In [15]:
# Explore test data:
print(test_ed.shape)
print(test_hh.shape)
assert(test_hh[test_hh['uid'].isin(test_ed['uid'])].shape[0] == 1334)
## No issues here. just use them as they are

(1334, 67)
(1334, 24)


In [16]:
# Remove columns in hh data that are useless:

In [17]:
train_hh = train_hh.drop(columns=['HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18'])
test_hh = test_hh.drop(columns=['HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18'])

In [18]:
# Remove columns in ed data that are useless

In [19]:
columns_to_keep = [
    'uid',
    'ED_Q01', 'ED_Q02', 'ED_Q03', 'ED_Q04', 'ED_Q05', 'ED_Q06', 'ED_Q07', 'ED_Q08', 'ED_Q09', 'ED_Q10', 'ED_Q11', 
    'ED_Q14', 'ED_Q15', 'ED_Q16', 'ED_Q17', 
    'ED_Q19',
    'ED_Q23',
    'ED_Q26', 'ED_Q27', 'ED_Q28', 'ED_Q29',
    'ED_Q41'
]

train_ed = train_ed[columns_to_keep]
test_ed = test_ed[columns_to_keep]

# Group #1 

Claim: Those who answer No = 2 for ED_Q3 are done with the ED survey. 


In [77]:
# get group_1 train sets
group1_train = train_ed['ED_Q03'].eq(2)
ed_group1_train = train_ed[group1_train]

ed_group1_train = ed_group1_train[['uid', 'ED_Q01', 'ED_Q02']]
hh_group1_train = train_hh[train_hh['uid'].isin(ed_group1_train['uid'])]


In [79]:
# get group1 test sets
group1_test = test_ed['ED_Q03'].eq(2)
ed_group1_test = test_ed[group1_test]

ed_group1_test = ed_group1_test[['uid', 'ED_Q01', 'ED_Q02']]
hh_group1_test = test_hh[test_hh['uid'].isin(ed_group1_test['uid'])]


In [86]:
# Save them to /group1
ed_group1_train.to_csv('./group1/ed_train_g1.csv', index=False)
hh_group1_train.to_csv('./group1/hh_train_g1.csv', index=False)
ed_group1_test.to_csv('./group1/ed_test_g1.csv', index=False)
hh_group1_test.to_csv('./group1/hh_test_g1.csv', index=False)

# Group 2

Where to split....

In [97]:
ed_g2_temp_train = train_ed.loc[~group1_train]


In [119]:
g2_bool = ed_g2_temp_train['ED_Q14'].eq(2)
ed_g2_temp_train  = ed_g2_temp_train[g2_bool]
ed_g2_temp_train

,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
196,678_7_3,1,1,1,1.0,9.0,2.0,0.0,2.0,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
197,687_3_2,1,1,1,2.0,3.0,3.0,0.0,2.0,NaN,...,NaN,NaN,12.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
198,179_7_2,1,1,1,2.0,3.0,3.0,1.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
199,833_8_2,1,1,1,5.0,4.0,6.0,0.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
200,134_4_2,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5329,574_5_1,1,1,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5330,618_2_1,2,3,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5331,155_4_1,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5332,475_5_1,1,1,1,1.0,8.0,2.0,2.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
# Try making the split on Q8 or Q14: people who did not enroll this year or last year. 
# Actually do it on attendance 9 and 15

# train_ed[g2_bool]

# split = ed_g2_temp_train['ED_Q08'].eq(2) | ed_g2_temp_train['ED_Q09'].eq(2) | ed_g2_temp_train['ED_Q15'].eq(2)
# ed_g2_temp_train[split]

# Try to literally look for those who have completed school: 
split = ed_g2_temp_train['ED_Q11'].eq(13) | ed_g2_temp_train['ED_Q17'].eq(13)
ed_g2_temp_train[split]   ## THAT"S MY GROuP 2 SON


,uid,ED_Q01,ED_Q02,ED_Q03,ED_Q04,ED_Q05,ED_Q06,ED_Q07,ED_Q08,ED_Q09,...,ED_Q15,ED_Q16,ED_Q17,ED_Q19,ED_Q23,ED_Q26,ED_Q27,ED_Q28,ED_Q29,ED_Q41
198,179_7_2,1,1,1,2.0,3.0,3.0,1.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
199,833_8_2,1,1,1,5.0,4.0,6.0,0.0,2.0,NaN,...,NaN,NaN,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
209,462_7_1,1,1,1,1.0,8.0,2.0,1.0,2.0,NaN,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
215,677_6_2,1,1,1,1.0,8.0,2.0,3.0,2.0,NaN,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
341,250_1_1,1,1,1,5.0,5.0,6.0,0.0,2.0,NaN,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5328,829_4_4,1,1,1,6.0,3.0,7.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5329,574_5_1,1,1,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5330,618_2_1,2,3,1,1.0,4.0,1.0,1.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5331,155_4_1,1,1,1,1.0,8.0,2.0,0.0,2.0,NaN,...,NaN,NaN,13.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# data = group2_temp[['ED_Q26', 'ED_Q27']]
# non_trivial = ~data['ED_Q26'].isna()
# data[non_trivial]


In [29]:
 # There might be a group 3 which splits from group2 at Q14? everything after 20 is 0?
group3 = group2_temp['ED_Q14'].is(2)

SyntaxError: invalid syntax (1387834729.py, line 2)

# Households

before you even split into groups, try your hand at usign households to "guess" more people's poverty score and increase the amount of data used

In [ ]:
poverty['fid'] = poverty['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 


In [ ]:
# The first division is by "primary sampling unit." It's like the first sample. 
# Members from the same family are in the same sample. So psh_hh is basically the family id.
# idcode is then the id of the individual..

# maybe you could average family responses into the observation with the poverty score... if poverty scores only has about one person from each family and there are more in the full data.


psu1 = train_ed[train_ed['ED_Psu']==1]
print(psu1.shape)
family_1_12 = psu1[psu1['ED_Hh']== 12]
family_1_12

In [ ]:
cond = poverty['fid'].eq("1_12")
poverty[cond]
# we could, for instance, set the poverty_scores of all three members of the family equal to each other. 
# HOWEVER, the questions is: how rich do YOU THINK YOU are? a kid with no allowance may think he is starving. 
# Even so, I think the variable: How rich does your family think you are? is good for training. 

In [ ]:
# No repeated fid's in the poverty_df. 
# So no two members of the family have an opinion on how poor/rich they are. 
# but if you are a kid and your dad says youre poor, you probably think you're poor??? there's so many vars
frequencies = poverty['fid'].value_counts()
# frequencies

In [ ]:
# are there any of the fid's in the test data also in the poverty scores? probably not, it would be super nice.
test_hh['fid'] = test_hh['uid'].apply(lambda uid : '_'.join([uid.split('_')[0], uid.split('_')[1]]) ) 
poverty[poverty['uid'].isin(test_hh['fid'])] # Nope

In [ ]:
# why are there some entries with the SAME idcodes and HOUSEHOLD to each other!????
train_ed[train_ed['hh']==10] ### psu_hh .... first houses into groups psu is first level division.
# could we continuate? psu_hh is family
hh10 = df_pov_scores[df_pov_scores['hh']==10]
hh10['poverty_score'].describe()

In [ ]:
plt.hist(hh10['poverty_score'], bins=5, color='blue', edgecolor='black', alpha=0.7)